# Task

Captura una o varias imágenes con monedas no solapadas. Tras visualizar la imagen, identifica de forma interactiva (por ejemplo haciendo clic en la imagen) una moneda de un valor determinado en la imagen (por ejemplo de 1€). Tras ello, la tarea se resuelve mostrando por pantalla el número de monedas y la cantidad de dinero presentes en la imagen. No hay restricciones sobre utilizar medidas geométricas o de color. ¿Qué problemas han observado?

<ins>Nota:</ins> Para establecer la correspondencia entre píxeles y milímetros, comentar que la moneda de un euro tiene un diámetro de 23.25 mm. la de 50 céntimos de 24.35, la de 20 céntimos de 22.25, etc.

<ins>Extras:</ins> Considerar que la imagen pueda contener objetos que no son monedas y/o haya solape entre las monedas. Demo en vivo.

El mundo real es muy variado, las imágenes no siempre se capturan con unas condiciones de iluminación tan buenas o controladas. Ejemplo con aplicación de variantes de umbralizados ofrecidas por OpenCV

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
# Load the image of the coins
task_1_coins = cv2.imread("./Images/Coins/tarea1.jpg")

# Convert the image to grayscale
task_1_grey = cv2.cvtColor(task_1_coins, cv2.COLOR_BGR2GRAY)

# Apply inverted OTSU thresholding
# This method automatically determines the optimal threshold value for binarization
ret, task_1_otsu = cv2.threshold(task_1_grey, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

#### Procesamiento
Dibujará solamente los contornos externos de las monedas en una imagen vacía y contará el número de monedas (círculos)

In [ ]:
# Find external contours (useful for detecting coin edges)
external_contours, external_hierarchy = cv2.findContours(task_1_otsu, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Convert the original image to RGB for visualization
task_1_coins_rgb = cv2.cvtColor(task_1_coins, cv2.COLOR_BGR2RGB)

# Draw the detected external contours in green on the original image
cv2.drawContours(task_1_coins_rgb, external_contours, -1, (0, 255, 0), 3)

# Display the original image with detected external contours
plt.subplot(121)
plt.axis("off")
plt.imshow(task_1_coins_rgb)
plt.title('Original with external contours')

# Create an empty image to draw the detected coins
coins_contours = np.zeros(task_1_coins.shape)
number_of_coins = 0  # Initialize a counter for the number of coins detected

# Iterate through each detected external contour
for contour in external_contours:
    # Calculate the area of the contour
    area = cv2.contourArea(contour)

    # Set a minimum size threshold to filter out small areas (noise)
    if area > 10:
        # Calculate the perimeter of the contour
        perimeter = cv2.arcLength(contour, True)

        # Find the minimum enclosing circle for the contour
        (cx, cy), radius = cv2.minEnclosingCircle(contour)

        # Determine if the contour represents a circle
        """
        The comparison is based on the following:
        Since the area of a circle is pi*r^2 and the perimeter is 2*pi*r,
        the area/perimeter ratio should be approximately r/2.
        The script compares whether the radius/2 from the minEnclosingCircle
        matches the calculated area/perimeter ratio.
        (This method detects circles, not just coins.)
        """
        area_perimeter_relation = area / perimeter
        if radius / 2 > area_perimeter_relation - radius / 5 and radius / 2 < area_perimeter_relation + radius / 5:
            # If the contour has enough points, fit an ellipse to it (optional)
            if contour.shape[0] > 5:
                elipse = cv2.fitEllipse(contour)

            # Draw the contour on the empty image for visualization
            cv2.drawContours(coins_contours, [contour], -1, (255, 255, 255), -1)

            # Increase the coin counter
            number_of_coins += 1

# Display the result image with detected coins and a title showing the count
plt.subplot(122)
plt.axis("off")
plt.title(f"{number_of_coins} coins detected.")
plt.imshow(coins_contours)
plt.show()

Como las imágenes originales son demasiado grandes, se procede a hacerles un reescalado, puesto que va a agilizar enormemente el desarrollo de la tarea

In [ ]:
# Load the image of the overlapped coins
task_2_overlapped_coins = cv2.imread("./Images/Coins/tarea2.jpg")

# Resize the first image (task_1_coins) by reducing its width and height by half
# Use INTER_AREA interpolation, which is good for shrinking images
cv2.imwrite("./Images/Coins/tarea_1_resized.jpg", 
            cv2.resize(task_1_coins, 
                       (task_1_coins.shape[1] // 2, task_1_coins.shape[0] // 2), 
                       interpolation = cv2.INTER_AREA))

# Resize the second image (task_2_overlapped_coins) similarly, reducing by half
cv2.imwrite("./Images/Coins/tarea_2_resized.jpg", 
            cv2.resize(task_2_overlapped_coins, 
                       (task_2_overlapped_coins.shape[1] // 2, task_2_overlapped_coins.shape[0] // 2), 
                       interpolation = cv2.INTER_AREA))

# Load the resized image of non-overlapped coins
non_overlapped_img = cv2.imread("./Images/Coins/tarea_1_resized.jpg")

# Load the resized image of overlapped coins
overlapped_img = cv2.imread("./Images/Coins/tarea_2_resized.jpg")

In [ ]:
def calculate_quantity(img, index, circles):
    # Get the radius of the reference euro coin (from the circle at the specified index)
    euro_radius = circles[index][2]
    
    # List of proportions of other coin radii relative to the euro coin
    proportion_to_euro = [0.6993548387096775, 0.8064516129032258, 0.9139784946236559, 
                          0.8494623655913979, 0.956989247311828, 1.043010752688172, 
                          1.0, 1.10752688172043]
    
    # Dictionary mapping proportions to their respective coin values in euros
    values = {
        0.6993548387096775: 0.01,  # 1 cent coin
        0.8064516129032258: 0.02,  # 2 cent coin
        0.9139784946236559: 0.05,  # 5 cent coin
        0.8494623655913979: 0.10,  # 10 cent coin
        0.956989247311828: 0.20,   # 20 cent coin
        1.043010752688172: 0.50,   # 50 cent coin
        1.0: 1.0,                  # 1 euro coin
        1.10752688172043: 2.0      # 2 euro coin
    }
    
    quantity = 0  # Initialize the total value of coins detected
    
    # Iterate over each detected circle (coin) in the image
    for circle in circles:
        x, y, r = circle  # Extract the x, y coordinates and radius of the current coin
        
        # Calculate the proportion of the current coin's radius to the euro coin's radius
        proportion = r / euro_radius
        
        key = 0  # Initialize key to store the closest matching proportion
        min_diff = 100000  # Initialize a large difference value to find the closest match
        
        # Iterate through the predefined proportions to find the closest match
        for v in proportion_to_euro:
            diff = abs(proportion - v)  # Calculate the difference from the current proportion
            if diff < min_diff:  # Update the key if this difference is smaller than the previous
                min_diff = diff
                key = v
        
        # Add the value of the detected coin to the total quantity
        quantity += values[key]
        
        # Annotate the image with the value of the detected coin near its location
        cv2.putText(img, f"{values[key]}", (int(x + r + 10), int(y)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, 2)
    
    # Print the total value of coins detected
    print(f"{quantity} euros detected.\n")
    
    return None

In [ ]:
def is_in_circle(x, y, circles):
    # Iterate through each circle in the list of circles along with its index
    for index, circle in enumerate(circles):
        c_x, c_y, r = circle  # Extract the center coordinates (c_x, c_y) and radius (r) of the current circle
        
        # Check if the point (x, y) is inside the current circle
        # This is done by calculating the squared distance from the point to the circle's center
        # and comparing it to the squared radius
        if (x - c_x)**2 + (y - c_y)**2 < r**2:
            return index  # Return the index of the circle if the point is inside it
    
    return None  # Return None if the point is not inside any circle

In [ ]:
def handle_click(x, y, img, circles):
    # Check if the clicked point (x, y) is inside any of the circles
    is_circle = is_in_circle(x, y, circles[0])  
    
    # If the point is inside a circle (is_circle is not None)
    if is_circle != None:
        # Calculate the quantity of coins based on the detected circle
        calculate_quantity(img, is_circle, circles[0])
        
        # Get the center coordinates and radius of the detected circle
        cx, cy, cr = circles[0][is_circle]
        
        # Draw the detected circle on the image in red with a thickness of 4
        cv2.circle(img, (int(cx), int(cy)), int(cr), (0, 0, 255), 4)
    
    # Return the modified image
    return img

In [ ]:
def get_and_draw_circles(img):
    # Convert the input image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply median blur to the grayscale image to reduce high-frequency noise
    pimg = cv2.medianBlur(gray_img, 7)

    # Use Hough Circle Transform to detect circles in the blurred image
    circles = cv2.HoughCircles(
        pimg,
        cv2.HOUGH_GRADIENT,  # Method for circle detection
        0.2,                  # Inverse ratio of the accumulator resolution to the image resolution
        15,                   # Minimum distance between detected circles
        param1=100,          # Higher threshold for the Canny edge detector
        param2=35,           # Accumulator threshold for the circle centers
        minRadius=5,         # Minimum radius of circles to be detected
        maxRadius=100        # Maximum radius of circles to be detected
    )

    # If circles are detected, draw them on the original image
    if circles is not None:
        for circle in circles[0]:
            x, y, r = circle  # Extract the x, y coordinates and radius of each detected circle
            cv2.circle(img, (int(x), int(y)), int(r), (0, 255, 0), 2)  # Draw the circle in green with thickness 2
    
    # Return the modified image and the list of detected circles
    return (img, circles)

Imagenes con círculos marcados y círculos detectados

In [ ]:
# Call the get_and_draw_circles function on a copy of the non-overlapped image
# This returns the image with detected circles drawn on it and the circles data
outlined_non_overlapped_img, non_overlapped_img_circles = get_and_draw_circles(non_overlapped_img.copy())

# Call the get_and_draw_circles function on a copy of the overlapped image
# This also returns the image with detected circles and the circles data
outlined_overlapped_img, overlapped_img_circles = get_and_draw_circles(overlapped_img.copy())

"""
# Display the results in a subplot to visualize the detected circles and calibrate Hough parameters
plt.subplot(121)  # Create a subplot for the non-overlapped image (left side)
plt.axis("off")   # Turn off the axis labels for a cleaner look
plt.imshow(outlined_non_overlapped_img)  # Show the outlined non-overlapped image with circles

plt.subplot(122)  # Create a subplot for the overlapped image (right side)
plt.axis("off")   # Turn off the axis labels for a cleaner look
plt.imshow(outlined_overlapped_img)  # Show the outlined overlapped image with circles
"""

In [ ]:
# Load the image from the specified path for the overlapped coins task
task_2_overlapped_coins = cv2.imread("./Images/Coins/tarea2.jpg")

# Resize the non-overlapped coins image to half its original size and save it
# The resize operation uses INTER_AREA interpolation for better quality when reducing size
cv2.imwrite("./Images/Coins/tarea_1_resized.jpg", cv2.resize(task_1_coins,
                                                             (task_1_coins.shape[1] // 2, task_1_coins.shape[0] // 2),  # New dimensions (width, height)
                                                             interpolation=cv2.INTER_AREA  # Use area interpolation for resizing
))

# Resize the overlapped coins image to half its original size and save it
cv2.imwrite("./Images/Coins/tarea_2_resized.jpg", cv2.resize(task_2_overlapped_coins,
                                                             (task_2_overlapped_coins.shape[1] // 2, task_2_overlapped_coins.shape[0] // 2),  # New dimensions (width, height)
                                                             interpolation=cv2.INTER_AREA  # Use area interpolation for resizing
))

# Load the resized non-overlapped coins image into a variable
non_overlapped_img = cv2.imread("./Images/Coins/tarea_1_resized.jpg")

# Load the resized overlapped coins image into a variable
overlapped_img = cv2.imread("./Images/Coins/tarea_2_resized.jpg")

In [ ]:
# Display the non-overlapped coins image with detected circles in a window titled "Monedas no solapadas"
cv2.imshow("Monedas no solapadas", outlined_non_overlapped_img)

# Display the overlapped coins image with detected circles in a window titled "Monedas solapadas"
cv2.imshow("Monedas solapadas", outlined_overlapped_img)

# Define a function to handle mouse click events for the non-overlapped coins image
def non_overlapped_mouse_click(event, x, y, flags, param):
    # Check if the left mouse button is pressed
    if event == cv2.EVENT_LBUTTONDOWN:
        # Handle the click using the handle_click function and update the image
        clicked_img = handle_click(x, y, outlined_non_overlapped_img.copy(), non_overlapped_img_circles)
        # Show the updated image in the same window
        cv2.imshow("Monedas no solapadas", clicked_img)

# Define a function to handle mouse click events for the overlapped coins image
def overlapped_mouse_click(event, x, y, flags, param):
    # Check if the left mouse button is pressed
    if event == cv2.EVENT_LBUTTONDOWN:
        # Handle the click using the handle_click function and update the image
        clicked_img = handle_click(x, y, outlined_overlapped_img.copy(), overlapped_img_circles)
        # Show the updated image in the same window
        cv2.imshow("Monedas solapadas", clicked_img)

# Assign the mouse callback function for the overlapped coins image window
cv2.setMouseCallback('Monedas solapadas', overlapped_mouse_click)

# Assign the mouse callback function for the non-overlapped coins image window
cv2.setMouseCallback('Monedas no solapadas', non_overlapped_mouse_click)

# Wait indefinitely until a key is pressed
cv2.waitKey(0)

# Close all OpenCV windows after a key press
cv2.destroyAllWindows()